In [8]:
import torch
from torchvision import transforms
from PIL import Image
from vit_pytorch import ViT

In [9]:
# Load the saved model
model = ViT(
    image_size=64,
    patch_size=4,
    num_classes=2,
    dim=128,
    depth=8,
    heads=8,
    mlp_dim=128,
    dropout=0.1,
    emb_dropout=0.1
)
model.load_state_dict(torch.load(r'C:\Users\dali_\OneDrive\Bureau\5edma\PPP\My_Ultimate_Model.pth'))

<All keys matched successfully>

In [10]:
# Set the model to evaluation mode
model.eval()

ViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=4, p2=4)
    (1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
    (2): Linear(in_features=48, out_features=128, bias=True)
    (3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (transformer): Transformer(
    (layers): ModuleList(
      (0): ModuleList(
        (0): PreNorm(
          (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (fn): Attention(
            (attend): Softmax(dim=-1)
            (dropout): Dropout(p=0.1, inplace=False)
            (to_qkv): Linear(in_features=128, out_features=1536, bias=False)
            (to_out): Sequential(
              (0): Linear(in_features=512, out_features=128, bias=True)
              (1): Dropout(p=0.1, inplace=False)
            )
          )
        )
        (1): PreNorm(
          (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)


In [11]:
# Define the data transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [12]:
# Load the test image
image_path = r"C:\Users\dali_\OneDrive\Bureau\5edma\PPP\DATA_LIDC_IDRI\CT_of_a_subpleural_nodule.png"
image = Image.open(image_path).convert('RGB')
image = transform(image).unsqueeze(0)  # Add batch dimension

In [13]:
# Make a prediction
with torch.no_grad():
    output = model(image)
    probabilities = torch.nn.functional.softmax(output, dim=1)
    _, predicted = torch.max(probabilities.data, 1)

In [14]:
if predicted.item() == 1:
    print('The test image contains a nodule with probability {:.2f}%.'.format(probabilities[0][1]*100))
else:
    print('The test image does not contain a nodule with probability {:.2f}%.'.format(probabilities[0][0]*100))

The test image contains a nodule with probability 99.39%.
